In [1]:
import time
import requests
import threading
from bs4 import BeautifulSoup

In [2]:
global car_detail_all
car_detail_all = []

## 详情页内容解析函数

In [3]:
#由详情页html解析其中的内容
#---------------------------
# 传入html总文件
# 解析出连接返回一个列表[]
#---------------------------

def get_detail_all(url):
    global car_detail_all
    
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'lxml')

    car_name = soup.find_all(class_="title")
    name = car_name[0].get_text()


    oldcar_price = soup.find_all(class_="newcar-price")
    o_price = oldcar_price[0].get_text()


    newcar_price = soup.find_all(class_="newcar-price-tax")
    n_price = newcar_price[0].get_text()


    car_summary = soup.find_all(class_="parameter")
    car_summary2 = []
    for i in car_summary:
        car_summary2.append(i.get_text())


    tag = soup.find_all(class_="tag")
    tag_all = []
    for i in tag:
        tag_all.append(i.get_text())


    detail = soup.find_all(class_="result")
    detail2 = detail[0].get_text()


    result = [name, o_price, n_price, "/".join(car_summary2), "/".join(tag_all), detail2]

    car_detail_all.append(result)
    return
    
#----------------------------------------------------------------    
    
def get_detail_all2(url):
    
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'lxml')

    car_name = soup.find_all(class_="title")
    name = car_name[0].get_text()
    print("车辆名称：", car_name[0].get_text())


    oldcar_price = soup.find_all(class_="newcar-price")
    o_price = oldcar_price[0].get_text()
    print("二手车售价：", oldcar_price[0].get_text())


    newcar_price = soup.find_all(class_="newcar-price-tax")
    n_price = newcar_price[0].get_text()
    print("新车价格：", newcar_price[0].get_text())


    car_summary = soup.find_all(class_="parameter")
    car_summary2 = []
    for i in car_summary:
        car_summary2.append(i.get_text())
    print("具体信息：", car_summary2)


    tag = soup.find_all(class_="tag")
    tag_all = []
    for i in tag:
        tag_all.append(i.get_text())
    print("标注信息：", tag_all)


    detail = soup.find_all(class_="result")
    detail2 = detail[0].get_text()
    print("检测报告：", detail2)


    result = [name, o_price, n_price, "/".join(car_summary2), "/".join(tag_all), detail2]
    #print(result)
    print()
    return result


## 读入链接数据

In [4]:
import pandas as pd
import numpy as np    
data = "data/car_url_new.csv"
pd_table = pd.read_csv(data, usecols=[1], header=None) # 没有表头
array_table = np.array(pd_table)
print(array_table[0:6])

[['car_url']
 ['/sh/car/75407afda78d8e21']
 ['/huizhou/car/14bca086b2ec9ea8']
 ['/sjz/car/73166ab39e82bd71']
 ['/bj/car/3c737d67ab496644']
 ['/nb/car/8f4bf512a977e50b']]


## 单次解析测试

In [5]:
#经测试，发现是静态html文件，这就比较好办了
#获得html文件所有内容

url = r"https://www.renrenche.com" + array_table[10000][0]
print(url)

result1 = get_detail_all2(url)
print(result1)

https://www.renrenche.com/bj/car/aa749b52cc6d7935
车辆名称： 起亚-起亚K5 2014款 2.0L 自动LUXURY
二手车售价： 6.00万
新车价格： 新车含税20.49万
具体信息： ['17.36万公里行驶里程', '8年3个月上牌', '北京车辆所在地', '国四外迁查询', '自动变速箱', '0次 过户记录', '7天内上架时间']
标注信息： ['超值', '急售', '0过户']
检测报告： 该车外观未发现钣金、色差， 但局部喷漆；外观多处有瑕疵，外观少量更换；灯光系统正常；内饰整洁；电子系统正常；发动机、变速箱工况正常，怠速规律无抖动，转向灵活；个人一手车；综合车况较好。无重大事故无泡水事故无火烧事故重大事故排查水泡车排查12项火烧车排查5项重大撞击排查17项轻微碰撞排查21项易损易耗部件23项功能部件检测内部配置15项动力、传动系统9项安全系统5项灯光、空调系统8项外观内饰检测内饰检测24项外观检测43项6项启动检测4项

['起亚-起亚K5 2014款 2.0L 自动LUXURY', '6.00万', '新车含税20.49万', '17.36万公里行驶里程/8年3个月上牌/北京车辆所在地/国四外迁查询/自动变速箱/0次 过户记录/7天内上架时间', '超值/急售/0过户', '该车外观未发现钣金、色差， 但局部喷漆；外观多处有瑕疵，外观少量更换；灯光系统正常；内饰整洁；电子系统正常；发动机、变速箱工况正常，怠速规律无抖动，转向灵活；个人一手车；综合车况较好。无重大事故无泡水事故无火烧事故重大事故排查水泡车排查12项火烧车排查5项重大撞击排查17项轻微碰撞排查21项易损易耗部件23项功能部件检测内部配置15项动力、传动系统9项安全系统5项灯光、空调系统8项外观内饰检测内饰检测24项外观检测43项6项启动检测4项']


## 多线程详情页

In [6]:
#多线程解析详情页情况
car_detail_all = []
thread_list = []


for k in range(0,1056,1):
    start = time.time()
    
    for i in range(1,51,1):
        url = r"https://www.renrenche.com" + array_table[50*k+i][0]
        thread = threading.Thread(target=get_detail_all, args=(url,))
        thread_list.append(thread)
        thread.start()

    for thread in thread_list:
        thread.join()
    
    thread_list = []
    
    end = time.time()
    print(50*k, f'访问50次网页，耗时：{end - start}')


Exception in thread Thread-18:
Traceback (most recent call last):
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\daichuan\AppData\Local\Temp/ipykernel_12592/2605260685.py", line 14, in get_detail_all
IndexError: list index out of range


0 访问50次网页，耗时：2.717994213104248
50 访问50次网页，耗时：2.7078402042388916
100 访问50次网页，耗时：2.595933198928833
150 访问50次网页，耗时：2.2978129386901855
200 访问50次网页，耗时：2.4247353076934814
250 访问50次网页，耗时：2.8109261989593506
300 访问50次网页，耗时：2.3119587898254395
350 访问50次网页，耗时：2.2211203575134277
400 访问50次网页，耗时：2.4551851749420166
450 访问50次网页，耗时：2.244598150253296
500 访问50次网页，耗时：2.282353162765503
550 访问50次网页，耗时：2.1708154678344727
600 访问50次网页，耗时：2.3255562782287598
650 访问50次网页，耗时：2.005275011062622
700 访问50次网页，耗时：2.1405234336853027
750 访问50次网页，耗时：2.1027989387512207
800 访问50次网页，耗时：2.4590461254119873
850 访问50次网页，耗时：2.1682677268981934
900 访问50次网页，耗时：2.3852832317352295
950 访问50次网页，耗时：2.3988282680511475
1000 访问50次网页，耗时：2.212796211242676
1050 访问50次网页，耗时：2.2006890773773193
1100 访问50次网页，耗时：2.295599937438965
1150 访问50次网页，耗时：2.4236083030700684
1200 访问50次网页，耗时：2.2279062271118164
1250 访问50次网页，耗时：2.127809524536133
1300 访问50次网页，耗时：2.4117352962493896
1350 访问50次网页，耗时：2.5882952213287354


Exception in thread Thread-1430:
Traceback (most recent call last):
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\daichuan\AppData\Local\Temp/ipykernel_12592/2605260685.py", line 14, in get_detail_all
IndexError: list index out of range


1400 访问50次网页，耗时：3.9345059394836426
1450 访问50次网页，耗时：2.0821781158447266
1500 访问50次网页，耗时：2.2724952697753906
1550 访问50次网页，耗时：2.0925331115722656
1600 访问50次网页，耗时：2.1779942512512207
1650 访问50次网页，耗时：2.0884668827056885
1700 访问50次网页，耗时：3.1644718647003174
1750 访问50次网页，耗时：2.322561502456665
1800 访问50次网页，耗时：2.861602783203125
1850 访问50次网页，耗时：2.3831961154937744
1900 访问50次网页，耗时：2.1545891761779785
1950 访问50次网页，耗时：2.192471981048584
2000 访问50次网页，耗时：2.1033380031585693
2050 访问50次网页，耗时：2.0709950923919678
2100 访问50次网页，耗时：2.12841796875
2150 访问50次网页，耗时：2.1635921001434326
2200 访问50次网页，耗时：2.1500203609466553
2250 访问50次网页，耗时：2.181612014770508
2300 访问50次网页，耗时：2.308771848678589
2350 访问50次网页，耗时：2.2696733474731445
2400 访问50次网页，耗时：2.3458945751190186
2450 访问50次网页，耗时：2.3639562129974365
2500 访问50次网页，耗时：2.1962296962738037


Exception in thread Thread-2605:
Traceback (most recent call last):
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\daichuan\AppData\Local\Temp/ipykernel_12592/2605260685.py", line 14, in get_detail_all
IndexError: list index out of range


...

51450 访问50次网页，耗时：2.4559292793273926


Exception in thread Thread-51526:
Traceback (most recent call last):
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\daichuan\AppData\Local\Temp/ipykernel_12592/2605260685.py", line 14, in get_detail_all
IndexError: list index out of range


51500 访问50次网页，耗时：2.4126861095428467
51550 访问50次网页，耗时：3.090080738067627
51600 访问50次网页，耗时：5.008420944213867
51650 访问50次网页，耗时：4.86374306678772
51700 访问50次网页，耗时：6.471642732620239
51750 访问50次网页，耗时：8.972453117370605
51800 访问50次网页，耗时：5.378248691558838
51850 访问50次网页，耗时：5.315564155578613
51900 访问50次网页，耗时：4.771252870559692
51950 访问50次网页，耗时：6.452820301055908
52000 访问50次网页，耗时：3.4255893230438232
52050 访问50次网页，耗时：3.1281681060791016


Exception in thread Thread-52156:
Traceback (most recent call last):
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\MYSOFT\anaconda\anaconda\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\daichuan\AppData\Local\Temp/ipykernel_12592/2605260685.py", line 14, in get_detail_all
IndexError: list index out of range


52100 访问50次网页，耗时：4.208495140075684
52150 访问50次网页，耗时：4.4721925258636475
52200 访问50次网页，耗时：2.5823519229888916
52250 访问50次网页，耗时：5.393230676651001
52300 访问50次网页，耗时：2.3468329906463623
52350 访问50次网页，耗时：2.097311496734619
52400 访问50次网页，耗时：2.518885374069214
52450 访问50次网页，耗时：2.397705316543579
52500 访问50次网页，耗时：2.112677574157715
52550 访问50次网页，耗时：2.128466844558716
52600 访问50次网页，耗时：2.0974767208099365
52650 访问50次网页，耗时：2.379310131072998
52700 访问50次网页，耗时：2.2272469997406006
52750 访问50次网页，耗时：2.242107629776001


## 写入文件

In [7]:
#写入csv文件
import pandas as pd

name=['car_name', 'two_price', 'new_price', 'summary', 'tag', 'Chinese_description']
test=pd.DataFrame(columns=name, data=car_detail_all)
test.to_csv('data/car_detail.csv', encoding='gbk')

print("写入完成",name)

写入完成 ['car_name', 'two_price', 'new_price', 'summary', 'tag', 'Chinese_description']
